In [1]:
import serial
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import butter, lfilter, sosfreqz, zpk2sos


# Set the correct serial port and baud rate
arduino_port = 'COM4'  # Change this to your Arduino port
baud_rate = 230400  # Should match the baud rate in the Arduino code

# Open the serial connection
ser = serial.Serial(arduino_port, baud_rate, timeout=2)

# Configuration
buffer_size = 100  # Should match BUFFER_SIZE in Arduino code
sampling_rate = 10000  # Adjust as per your Arduino code configuration
channels = 1  # Should match numberOfChannels in Arduino code

# Initialize data buffer
data_buffer = np.zeros((buffer_size, channels))

# Create a time array for x-axis
time_array = np.arange(0, buffer_size / sampling_rate, 1 / sampling_rate)

# Filtering setup
lowcut = 1.0  # Low-pass cutoff frequency in Hz
highcut = 50.0  # High-pass cutoff frequency in Hz
notch_frequency = 60.0  # Notch frequency in Hz

nyq = 0.5 * sampling_rate
low = lowcut / nyq
high = highcut / nyq
notch = notch_frequency / nyq

# Design filters
lowpass_b, lowpass_a = butter(4, low, btype='low', analog=False)
highpass_b, highpass_a = butter(4, high, btype='high', analog=False)
notch_b, notch_a = sosfreqz(*zpk2sos(*zpk(2 * np.pi * notch_frequency, nyq)))

# Plot setup
plt.ion()  # Enable interactive mode for real-time plotting
fig, ax = plt.subplots()

# Main loop
try:
    while True:
        # Read a line of data from the serial port
        line = ser.readline().decode().strip()

        # Process the received data
        if line:
            # Assuming data is formatted as "MSB LSB MSB LSB MSB LSB ..."
            raw_data = [int(line[i:i + 2], 16) for i in range(0, len(line), 2)]

            # Assuming two channels, modify as per your Arduino code
            data_buffer[:-1] = data_buffer[1:]
            data_buffer[-1] = raw_data[1::2]

            # Apply filters
            lowpass_filtered = lfilter(lowpass_b, lowpass_a, data_buffer, axis=0)
            highpass_filtered = lfilter(highpass_b, highpass_a, lowpass_filtered, axis=0)
            filtered_data = sosfreqz(notch_b, notch_a, highpass_filtered, axis=0)

            # Plot the filtered data
            ax.clear()
            for i in range(channels):
                ax.plot(time_array, filtered_data[:, i], label=f'Channel {i + 1}')
            ax.legend()
            ax.set_xlabel('Time (s)')
            ax.set_ylabel('Filtered Signal')
            plt.pause(0.01)  # Pause for a short time to allow the plot to update

except KeyboardInterrupt:
    print("Exiting...")

finally:
    # Close the serial connection
    ser.close()
    plt.ioff()  # Disable interactive mode
    plt.show()


NameError: name 'zpk' is not defined